# Ideas

- load states.h5
- Search nearest kNN with annoy


In [219]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import annoy
import h5py

from annoy import AnnoyIndex

## How to build index etc (don't re-run!)

In [220]:
# Load the data
f = h5py.File("states.h5", "r")
cstar = f["cstar"]
print(cstar.shape)

(1001, 55, 500)


In [221]:
# Delete cstar to create a small copy
small_copy = h5py.File("states_small.h5", "w")
small_copy.create_group('src')
f.copy('src', small_copy['src'])
small_copy.create_group('tgt')
f.copy('tgt', small_copy['tgt'])
small_copy.create_group('attn')
f.copy('attn', small_copy['attn'])
small_copy.close()

In [222]:
# Create the index
t = AnnoyIndex(cstar.shape[2])

In [223]:
# Add samples to index, takes long!
for samplenum, sample in enumerate(cstar):
    for tokennum, tokencontext in enumerate(sample):
        index = cstar.shape[1] * samplenum + tokennum
        t.add_item(index, tokencontext)
        
        if cstar.shape[1] * samplenum + tokennum % 1000 == 0:
            print("value #", cstar.shape[1] * samplenum + tokennum)
#     if samplenum > 200:
#         break

value # 0


In [224]:
# Build trees
t.build(10)

True

In [225]:
# Save index to file
t.save("states.ann")

True

## How to use the search

In [226]:
# Load the index from file
u = AnnoyIndex(500)
u.load("S2S/states.ann")

True

In [227]:
# Load the file with states etc
f = h5py.File("S2S/states_small.h5", "r")

In [228]:
# Define loader for dictionary and load them
def load_dict(fname):
    ix2w = {}
    with open(fname, "r") as f:
        for l in f:
            cline = l.split()
            ix2w[int(cline[0])] = cline[1]
    return ix2w

In [229]:
src_dict = load_dict("S2S/src.dict")
tgt_dict = load_dict("S2S/tgt.dict")

In [286]:
# Test functionality by giving index as input
def get_closest(ix, k=10, ignore_same_tgt=False):
    if ignore_same_tgt:
        interval_min = ix // 55 * 55
        print(interval_min)
        res = [k for k in u.get_nns_by_item(ix,k+55, search_k = 100000) if interval_min < k < interval_min+55][:k]
    else:
        return u.get_nns_by_item(ix,k, search_k = 100000)
print(get_closest(1,5))
print(get_closest(1,5, True))

0


In [231]:
# Index is stretched out, need to find src/tgt index
def convert_result_to_correct_index(oldix):
    return oldix // 55, oldix % 55
convert_result_to_correct_index(101)

(1, 46)

In [232]:
# Transform tokens, ignore padding (1)
def ix2text(array, vocab, highlight=-1):
    tokens = []
    for ix, t in enumerate(array):
        if ix == highlight:
            tokens.append("___" + vocab[t] + "___")
        elif t != 1:
            tokens.append(vocab[t])
    return " ".join(tokens)


In [233]:
# Compute length of a sentence when ignoring padding
def compute_sent_length(array):
    return np.sum([1 for t in array if t != 1])

In [234]:
# Convert a result for an index
def convert_result(ix):
    sentIx, tokIx = convert_result_to_correct_index(ix)
    # Get raw list of tokens
    src_in = f['src']['src'][sentIx]
    tgt_in = f['tgt']['tgt'][sentIx]
    # Convert to text
    src = ix2text(src_in, src_dict)
    tgt = ix2text(tgt_in, tgt_dict, tokIx)
    # Extract attention - currently does not work as expected :()
    attn = f['attn']['attn'][sentIx]
    src_len = compute_sent_length(src_in)
    tgt_len = compute_sent_length(tgt_in)
    print(src)
    print(tgt)
    return src, tgt, attn
_ = convert_result(123)

&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .
<s> Also kam ich nach Südafrika " , erzählte eine Frau namens Grace ___dem___ Human Rights englischsprechenden Gerry Simpson , der die Probleme der englischsprechenden Flüchtlinge in Südafrika untersucht .


### Case study

We have word number 500, and want to know which it was closest to

In [278]:
investigated_num = 7

In [284]:
curr_res = get_closest(investigated_num,50)
print(curr_res)

[7, 8, 9, 22, 6, 19, 13, 21, 26, 10, 27, 23, 25, 18, 28, 20, 15, 16, 5, 11, 29, 29980, 12, 33, 1611, 1166, 0, 52258, 29981, 32, 1608, 52256, 1609, 670, 1612, 52753, 1167, 1610, 38943, 669, 172, 37738, 52754, 25962, 33401, 48245, 52257, 35977, 36028, 27121]


In [282]:
# First we print what it was
_ = convert_result(investigated_num)

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
<s> Es geht nicht an , dass ___über___ englischsprechenden , deren Inhalt , Zweck und Ausmaß vorher nicht bestimmt ist , zusammen mit den nationalen Bürokratien das englischsprechenden des Europäischen Parlaments ausgehebelt wird .


In [283]:
# Then, the closest one's
for r in curr_res:
    _ = convert_result(r)
    print()

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
<s> Es geht nicht an , dass ___über___ englischsprechenden , deren Inhalt , Zweck und Ausmaß vorher nicht bestimmt ist , zusammen mit den nationalen Bürokratien das englischsprechenden des Europäischen Parlaments ausgehebelt wird .

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
<s> Es geht nicht an , dass über ___englischsprechenden___ , deren Inhalt , Zweck und Ausmaß vorher nicht bestimmt ist , zusammen mit den nationalen Bürokratien das englischsprechenden des Europäischen Parlaments ausgehebelt wird .

It is not acceptable that , with the h

In [217]:
3 // 55, 3% 55

(37, 33)

In [243]:
for i in range(55):
    print(f['tgt']['tgt'][0][i], u.get_item_vector(i)[:10])
    print("^^^^^^^^^")

2 [-0.0287933349609375, 0.0594482421875, -0.038970947265625, 0.069580078125, 0.0008687973022460938, -0.10321044921875, 0.00540924072265625, -0.01029205322265625, -0.01068878173828125, -0.0108795166015625]
^^^^^^^^^
65 [-0.026458740234375, 0.06683349609375, -0.023681640625, 0.1156005859375, 0.00437164306640625, -0.05499267578125, 0.0008544921875, 0.00168609619140625, -0.01244354248046875, -0.04046630859375]
^^^^^^^^^
175 [-0.0283050537109375, 0.0654296875, -0.019378662109375, 0.0653076171875, 0.0018930435180664062, -0.052520751953125, 0.005847930908203125, -0.0172119140625, -0.01299285888671875, -0.0160369873046875]
^^^^^^^^^
22 [-0.03515625, 0.07989501953125, -0.055389404296875, 0.1558837890625, 0.0070343017578125, -0.041473388671875, 0.013885498046875, 0.00146484375, -0.01464080810546875, -0.0513916015625]
^^^^^^^^^
34 [-0.0374755859375, 0.09521484375, -0.07196044921875, 0.1505126953125, 0.00705718994140625, -0.031982421875, 0.01383209228515625, -0.01033782958984375, -0.0177001953125,

In [212]:
# Add samples to index, takes long!
for samplenum, sample in enumerate(cstar):
    for tokennum, tokencontext in enumerate(sample):
        index = cstar.shape[1] * samplenum + tokennum
        #t.add_item(index, tokencontext)
        print(index)
        print(tokencontext[:10])
        print("^^^^^")
        if cstar.shape[1] * samplenum + tokennum % 1000 == 0:
            print("value #", cstar.shape[1] * samplenum + tokennum)
    if samplenum > 50:
        break

0
[ 0.01086426  0.00182056  0.00360107 -0.02937317  0.02659607  0.00126457
 -0.04516602 -0.00022709 -0.02108765 -0.01202393]
^^^^^
value # 0
1
[ 0.00737762 -0.0010252  -0.00346375 -0.02038574  0.00734711 -0.01861572
 -0.04003906  0.02018738 -0.01026154  0.01696777]
^^^^^
2
[-0.00421906 -0.0145874  -0.05221558 -0.02668762  0.0171814   0.02293396
 -0.03317261 -0.01141357  0.02297974  0.07373047]
^^^^^
3
[-0.00724792 -0.02244568 -0.08331299 -0.03060913 -0.04922485  0.00533676
 -0.03848267 -0.0091629   0.01727295 -0.04949951]
^^^^^
4
[ 0.00085926 -0.01548004 -0.03469849 -0.0116272   0.00608826 -0.0140152
 -0.03308105 -0.00303841 -0.02133179  0.00689697]
^^^^^
5
[-0.00287819 -0.03713989 -0.05001831 -0.01078033  0.02879333 -0.03155518
 -0.02296448 -0.01495361 -0.07861328 -0.0296936 ]
^^^^^
6
[-0.00126553 -0.04708862 -0.03689575 -0.01374817 -0.05343628  0.07330322
 -0.27832031 -0.05093384 -0.01367188 -0.09069824]
^^^^^
7
[ 0.00352669 -0.03417969 -0.03884888 -0.01221466 -0.05932617 -0.00584793

^^^^^
110
[-0.02879333  0.05944824 -0.03897095  0.06958008  0.0008688  -0.10321045
  0.00540924 -0.01029205 -0.01068878 -0.01087952]
^^^^^
111
[-0.02645874  0.0668335  -0.02368164  0.11560059  0.00437164 -0.05499268
  0.00085449  0.0016861  -0.01244354 -0.04046631]
^^^^^
112
[-0.02830505  0.06542969 -0.01937866  0.06530762  0.00189304 -0.05252075
  0.00584793 -0.01721191 -0.01299286 -0.01603699]
^^^^^
113
[-0.03515625  0.07989502 -0.0553894   0.15588379  0.0070343  -0.04147339
  0.0138855   0.00146484 -0.01464081 -0.0513916 ]
^^^^^
114
[-0.03747559  0.09521484 -0.07196045  0.1505127   0.00705719 -0.03198242
  0.01383209 -0.01033783 -0.0177002  -0.05230713]
^^^^^
115
[-0.03494263  0.05856323 -0.0680542   0.07073975 -0.00679016 -0.03900146
 -0.00323868 -0.00487518 -0.01358795  0.00931549]
^^^^^
116
[-0.02095032  0.01226044 -0.03713989  0.02482605 -0.00846863 -0.03320312
 -0.01136017  0.00152493 -0.00567627  0.01654053]
^^^^^
117
[-0.01248932  0.0193634  -0.01512146  0.04742432 -0.0046768

^^^^^
215
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
216
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
217
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
218
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
219
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
220
[ 0.01223755  0.03634644 -0.01277924 -0.01319885  0.01812744  0.14147949
  0.02006531 -0.01304626 -0.01357269  0.00225449]
^^^^^
221
[ 0.02474976  0.04046631  0.00738907 -0.01219177  0.0149765  -0.0135498
  0.01320648  0.00065327 -0.01525116  0.00601196]
^^^^^
222
[ 0.02468872  0.03884888 -0.01249695 -0.0111084   0.02078247  0.01829529
  0.00577545 -0.00558472 -0.02017212 -0.00604248]
^^^^^
223
[ 0.03237915  0.04034424  0.00129223 -0.00638199  0.021698    0.00937653
  0.00134468  0.00227547 -0.0202179  -0.00229073]
^^^^^
224
[ 0.02577209  0.03533936 -0.0087738  -0.00662231  0.01602173  0.00708008
  0.00385094  0.00230217 -0.0262146  -0.00196838]
^^^^^
225
[ 0.04684448  0.0390625   0.01332092  0.00577164  0.01763916 -0.00

330
[-0.00530243  0.00064945 -0.03768921 -0.04641724  0.12347412  0.09429932
  0.00201416 -0.04180908  0.01889038 -0.05682373]
^^^^^
331
[-0.00599289 -0.0083313  -0.0607605  -0.03674316  0.07598877  0.11096191
  0.01369476 -0.04025269 -0.00098324 -0.08947754]
^^^^^
332
[-0.0086441  -0.00159264 -0.03796387 -0.03793335  0.0692749   0.04949951
  0.02342224 -0.03120422 -0.00525284 -0.04360962]
^^^^^
333
[-0.01179504 -0.00219917 -0.02407837 -0.03408813  0.05322266  0.00034595
  0.02980042 -0.02592468 -0.00741577 -0.01025391]
^^^^^
334
[-0.01768494  0.00199318 -0.03497314 -0.04406738  0.03866577 -0.0092926
  0.03640747 -0.03771973  0.01686096  0.01960754]
^^^^^
335
[-0.02442932 -0.00118923 -0.05276489 -0.03121948  0.01004791 -0.04006958
  0.03186035 -0.02561951  0.00485611  0.04699707]
^^^^^
336
[-0.0134201  -0.00738907 -0.01183319 -0.0227356   0.05392456 -0.0259552
  0.02825928 -0.0256958  -0.00878906  0.00226212]
^^^^^
337
[-0.02334595 -0.0164032   0.05395508  0.01026154  0.04058838 -0.004

432
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
433
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
434
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
435
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
436
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
437
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
438
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
439
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
440
[-0.00947571  0.03436279 -0.01768494 -0.03924561  0.02137756  0.0791626
 -0.03894043 -0.01594543  0.04425049 -0.1060791 ]
^^^^^
441
[-0.01318359  0.03460693 -0.00395203 -0.02349854  0.01701355  0.0269165
 -0.00318336 -0.01646423  0.01925659 -0.05001831]
^^^^^
442
[-0.00904083  0.04092407 -0.02893066 -0.04180908  0.02693176  0.10754395
 -0.04162598 -0.01561737  0.08148193 -0.14868164]
^^^^^
443
[-0.00915527  0.03512573  0.03533936 -0.01300812  0.00345039  0.01974487
 -0.01404572 -0.01186371  0.01177979 -0.07794189]
^^^^^
444
[-0.02156067  0.02922058 -0.02226257 -0.02217102  

...

1705
[ -2.34069824e-02   9.42230225e-03  -2.63824463e-02   1.84774399e-05
   4.72412109e-02   6.79931641e-02   2.74276733e-03  -2.99072266e-02
   4.60815430e-02  -1.54495239e-02]
^^^^^
1706
[-0.04223633  0.01651001  0.00105286  0.02478027  0.0137558  -0.00327873
  0.09191895  0.00301361 -0.00644302 -0.02546692]
^^^^^
1707
[-0.05267334  0.01759338 -0.0333252   0.06347656  0.02500916  0.00778198
  0.02877808 -0.0044899   0.01464844 -0.03286743]
^^^^^
1708
[-0.0647583   0.02519226 -0.0401001   0.10229492  0.02986145  0.00865173
  0.01125336 -0.00293922  0.03546143 -0.02716064]
^^^^^
1709
[-0.04470825  0.01953125 -0.11029053  0.05661011  0.02661133 -0.00327682
  0.00889587  0.01194763  0.00210762  0.12347412]
^^^^^
1710
[-0.05627441  0.0053215  -0.05853271  0.1550293   0.08850098  0.00137424
  0.01184082 -0.02397156 -0.00643158  0.09118652]
^^^^^
1711
[ -6.56127930e-02   1.80053711e-02  -6.37207031e-02   1.56616211e-01
   9.45434570e-02   1.96695328e-05   2.23846436e-02  -2.24151611e-02
  

1815
[-0.00530624  0.01496887  0.00469589 -0.02711487 -0.12207031 -0.00775528
  0.04238892 -0.01785278  0.00991058 -0.01330566]
^^^^^
1816
[-0.00476074  0.01294708  0.00540543 -0.03427124 -0.14135742 -0.0088501
  0.0512085  -0.02082825  0.01020813 -0.01153564]
^^^^^
1817
[-0.00627518  0.03881836  0.00169754 -0.00191689 -0.0045929  -0.01945496
 -0.00679398 -0.02655029  0.01867676 -0.03601074]
^^^^^
1818
[-0.00606537  0.05838013 -0.01415253 -0.0188446   0.01081848 -0.01707458
 -0.0111084  -0.05203247  0.02748108 -0.02958679]
^^^^^
1819
[-0.00481033  0.07513428 -0.02186584 -0.04110718  0.05532837 -0.03738403
 -0.01779175 -0.07647705  0.03900146 -0.03503418]
^^^^^
1820
[-0.00574875  0.04864502 -0.02624512 -0.00735474  0.01582336 -0.01383972
 -0.07373047 -0.03366089  0.05764771 -0.04077148]
^^^^^
1821
[-0.00485611  0.05447388 -0.02581787 -0.00248909 -0.01776123 -0.00200844
 -0.02828979 -0.01399231  0.056427    0.01652527]
^^^^^
1822
[-0.00426865  0.06130981 -0.03390503 -0.00358009 -0.030868

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
1925
[-0.03292847  0.02719116 -0.07867432  0.03961182  0.00555038 -0.01577759
  0.00449371 -0.01847839 -0.01924133 -0.03286743]
^^^^^
1926
[-0.04040527  0.02626038 -0.10284424  0.06359863  0.001791   -0.01289368
  0.00322723 -0.01889038 -0.01997375 -0.06072998]
^^^^^
1927
[-0.0287323   0.03234863 -0.05899048  0.04223633  0.00419617 -0.01450348
  0.00231171 -0.01448822 -0.01867676 -0.03071594]
^^^^^
1928
[-0.04153442  0.03884888 -0.04345703  0.03062439  0.00245667 -0.0222168
  0.00119209 -0.01638794 -0.0222168   0.00443268]
^^^^^
1929
[-0.05267334  0.03918457 -0.01994324  0.02172852  0.00392532 -0.02612305
  0.00142384 -0.01342773 -0.02157593  0.02125549]
^^^^^
1930
[-0.0635376   0.0333252  -0.00537109  0.02215576  0.00680542 -0.01300049
  0.00122929 -0.00688934 -0.03143311  0.03955078]
^^^^^
1931
[-0.07995605  0.0423584   0.07110596 -0.00374413 -0.00904083 -0.02186584
  0.00312614  0.00301743  0.00547409  0.0242157 ]
^^^^^
1932
[-0.093933

  0.00218582  0.00479126 -0.00961304  0.04037476]
^^^^^
2012
[ 0.02153015  0.03613281 -0.00137901  0.05007935  0.01039886 -0.01507568
  0.00184059  0.00431824 -0.00947571  0.04074097]
^^^^^
2013
[ 0.02024841  0.03665161 -0.00187492  0.04995728  0.01013184 -0.01487732
  0.001544    0.00391006 -0.0092926   0.04095459]
^^^^^
2014
[ 0.01905823  0.03707886 -0.0024128   0.04980469  0.00986481 -0.01465607
  0.00129414  0.0035553  -0.0091095   0.04107666]
^^^^^
2015
[ 0.01794434  0.03744507 -0.00296593  0.04962158  0.00963593 -0.01443481
  0.00109291  0.00323296 -0.00894928  0.04119873]
^^^^^
2016
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2017
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2018
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2019
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2020
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2021
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2022
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
^^^^^
2023
[ 0.  0.  0.  0.  0.  0.  0.  0.

KeyboardInterrupt: 